In [2]:
from pathlib import Path
import ipywidgets as widgets
import anki2roam
from anki import Collection

#upload_suggest = widgets.Label(value="Use above button to upload your data file if not listed among the choices below.)")
#select_markdown = widgets.HTML(value=markdown.markdown("""Use above button to upload your data file if not among the choices you see listed below.<br/>**Choose the file with the data to plot from below**:""")) # based on https://github.com/jupyter-widgets/ipywidgets/issues/2428#issuecomment-500084610 and https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#HTML ; need to pip install markdown first and import it

out = widgets.Output()

# @out.capture(clear_output=True,wait=True)
# def on_button_clicked(b):
#     global file_uploader,select_file, current_file, ax
#     # get data file to be used
#     current_file = select_file.value
#     print(f"Using data from:\t{current_file}")
#     from ipywidgets import interact
#     from ipywidgets.widgets import FloatSlider, IntSlider, Dropdown
#
#     view_widget = Dropdown(options=v_angles)
#     elevation_widget= IntSlider(value=30, min=0, max=360,step=5, continuous_update=False)
#     azimuth_widget= IntSlider(value=185, min=0, max=360,step=5, continuous_update=False)
#     def update_elevation_and_azimuth(*args):
#         view_nums = view_widget.value.split(",")
#         view_nums = [int(x) for x in view_nums]
#         elevation_widget.value, azimuth_widget.value = view_nums
#     view_widget.observe(update_elevation_and_azimuth, 'value')
#
#     def plot_data(opacity, cmap, coloring,marker_size,view_choice,elevation,azimuth):
#         print(azimuth)
#
#     p = interact(plot_data,
#                  opacity=FloatSlider(value=0.75, min=0, max=1.0, step=0.05, readout_format='.3f', continuous_update=False),
#                  cmap=Dropdown(value='magma'),
#                  coloring=Dropdown(value='alcohol',options=['residual sugar','fixed acidity','alcohol']),
#                  marker_size= IntSlider(value=50, min=10, max=180,step=10, continuous_update=False),
#                  view_choice=view_widget,
#                  elevation= elevation_widget,
#                  azimuth= azimuth_widget
#                 )
#     vbox.children = [file_uploader, select_markdown, select_file,button, out]

# button.on_click(on_button_clicked)

description = widgets.HTML(value="Use the above 'Upload' button to "
                               "upload your data file if it is not among the choices you see "
                               "listed below.<br/><b>Click on the file name among list below "
                               "to select the data to plot</b> and then click the "
                               "'Plot Selected' button.")


file_uploader = widgets.FileUpload()
out_text = widgets.Textarea(value="testup2")
out_text.layout.width = '50%'

select_deck = widgets.Select(options=[], description='Decks:')
select_deck.layout.width = '50%'

collection_fname = "collection.anki2"
# collection = None

@out.capture()
def show_decks(change):
    out.clear_output()

    filename = next(iter(change['new']))
    Path(collection_fname).write_bytes(change['new'][filename]['content'])

    collection = Collection(collection_fname, log=True)
    select_deck.options=tuple(collection.decks.allNames())
    collection.close()

@out.capture()
def export_deck(button):
    exporter = anki2roam.MarkdownExporter(select_deck.value, ".")
    out_text.value = exporter.export_text()

file_uploader.observe(show_decks, 'value')

export_button = widgets.Button(description="Export selected deck")
export_button.on_click(export_deck)

vbox = widgets.VBox([description, file_uploader, select_deck,export_button, out_text, out])
vbox

